In [6]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import numpy as np
import os
import json

# Load MoveNet model
movenet = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")

# Define dataset paths
DATASET_PATH = r"C:\Users\debar\Downloads\dataset_yoga_model\dataset_yoga_model"

TRAIN_PATH = os.path.join(DATASET_PATH, "train")
TEST_PATH = os.path.join(DATASET_PATH, "test")

# Yoga pose classes
POSE_CLASSES = ["warrior", "tree", "dog", "no_pose", "shoulder_stand", "triangle", "cobra", "chair"]

# Function to extract keypoints using MoveNet
def extract_keypoints(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    input_image = cv2.resize(image, (192, 192))  # Resize for MoveNet
    input_image = np.expand_dims(input_image, axis=0)  # Add batch dimension
    input_image = tf.convert_to_tensor(input_image, dtype=tf.int32)  # Convert to int32

    # Run MoveNet model
    outputs = movenet.signatures["serving_default"](input_image)
    keypoints = outputs["output_0"].numpy().reshape(17, 3)  # 17 keypoints (x, y, confidence)

    return keypoints[:, :2]  # Only return (x, y) coordinates, ignore confidence

# Function to process dataset and save keypoints
def process_dataset(data_path, save_file):
    keypoint_data = []
    labels = []

    for pose_class in POSE_CLASSES:
        pose_path = os.path.join(data_path, pose_class)
        for image_name in os.listdir(pose_path):
            image_path = os.path.join(pose_path, image_name)
            keypoints = extract_keypoints(image_path)
            keypoint_data.append(keypoints.flatten())  # Flatten keypoints into a single array
            labels.append(POSE_CLASSES.index(pose_class))  # Convert class to index

    # Save as NumPy arrays
    np.save(f"{save_file}_keypoints.npy", np.array(keypoint_data))
    np.save(f"{save_file}_labels.npy", np.array(labels))

    print(f"✅ Processed {len(labels)} images from {data_path}")

# Process train and test datasets
process_dataset(TRAIN_PATH, "train")
process_dataset(TEST_PATH, "test")


✅ Processed 2139 images from C:\Users\debar\Downloads\dataset_yoga_model\dataset_yoga_model\train
✅ Processed 1011 images from C:\Users\debar\Downloads\dataset_yoga_model\dataset_yoga_model\test


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import jaccard_score

# Load extracted keypoints and labels
X_train = np.load("train_keypoints.npy")
y_train = np.load("train_labels.npy")
X_test = np.load("test_keypoints.npy")
y_test = np.load("test_labels.npy")

# Normalize keypoints (scale between 0 and 1)
X_train = X_train / np.max(X_train)
X_test = X_test / np.max(X_test)

# Convert labels to one-hot encoding
num_classes = len(np.unique(y_train))
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

# Build the CNN Model
model = Sequential([
    Dense(128, activation='relu', input_shape=(34,)),  # 17 keypoints * 2 (x, y)
    BatchNormalization(),
    Dropout(0.3),
    
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    
    Dense(32, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    
    Dense(num_classes, activation='softmax')  # Output layer
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=30, batch_size=32)

# Evaluate the model
train_loss, train_acc = model.evaluate(X_train, y_train)
test_loss, test_acc = model.evaluate(X_test, y_test)

print(f"✅ Training Accuracy: {train_acc:.2f}")
print(f"✅ Validation Accuracy: {test_acc:.2f}")

# Compute IoU Score
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

iou_score = jaccard_score(y_test_classes, y_pred_classes, average='macro')
print(f"✅ IoU Score: {iou_score:.2f}")


Epoch 1/30


c:\Users\debar\OneDrive\Documents\proj\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


67/67 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.2970 - loss: 2.0692 - val_accuracy: 0.4036 - val_loss: 1.8009
Epoch 2/30
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6677 - loss: 1.0381 - val_accuracy: 0.4916 - val_loss: 1.4719
Epoch 3/30
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7541 - loss: 0.7597 - val_accuracy: 0.6588 - val_loss: 1.1034
Epoch 4/30
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7902 - loss: 0.6542 - val_accuracy: 0.7349 - val_loss: 0.8553
Epoch 5/30
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8208 - loss: 0.5803 - val_accuracy: 0.8882 - val_loss: 0.5216
Epoch 6/30
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8141 - loss: 0.5229 - val_accuracy: 0.9426 - val_loss: 0.3452
Epoch 7/30
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8456 - loss: 0.4752 - val_accuracy: 0.8932 - val_loss: 0.3777
Epoch 8/30
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8502 - loss: 0.4845 - val_accuracy: 0.9278 - val_loss: 0.2663
Epo

In [1]:
import numpy as np
import tensorflow as tf
import sklearn

print("NumPy version:", np.__version__)
print("TensorFlow version:", tf.__version__)
print("Scikit-Learn version:", sklearn.__version__)



NumPy version: 1.26.4
TensorFlow version: 2.18.0
Scikit-Learn version: 1.6.1
